# Практическое задание к уроку 8. 
Рекуррентные нейронные сети RNN LSTM GRU
Данные берем отызывы за лето

На вебинаре мы говорили, что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для нашей задачи

построить свёрточные архитектуры

построить различные архитектуры с RNN

построить совместные архитектуры CNN -> RNN или (RNN -> CNN)

In [1]:
# попробуем запрограммировать простую рекурентную сеть. Возьмем датасет с прошлого занятия

import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re


In [2]:
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
df = pd.read_excel("../data/Otzyvy.xls")

In [4]:
df.head(2)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [6]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [7]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Flatten, Reshape
from keras.layers import Conv1D, GlobalMaxPool1D, MaxPooling1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [8]:
df['processed'] = df['Content'].apply(preprocess_text)

In [9]:
df.head(10)

,Rating,Content,Date,processed
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро
5,5,Всё удобно норм 👍👍👍,2017-08-14,удобно норма 👍👍👍
6,5,Очень удобное приложение.,2017-08-14,удобный приложение
7,5,Все устраивает,2017-08-14,устраивать
8,5,У меня работает все четко. В отличии от банком...,2017-08-14,работать чётко отличие банкомат вечно зависать...
9,5,Очень все хорошо👍,2017-08-14,хорошо👍


In [10]:
df['Rating'].value_counts()

Rating
5    14586
1     2276
4     2138
3      911
2      748
Name: count, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

**adjustment after several tries**   
- *Epochs =5 with early_stopping. With less epochs accuracy on validation (built-in model validation of 10%) after 3rd to 10 epochs begins to drop, while on train corpus loss continues to fall and accuracy to rise which seemes to indicate overlearning, BUT on test lower total accuracy gives all classes (even minor) prediction better, whereas if we stop at the lowest loss on validation, we get only three biggest classes TRUE-predictions at all*   
- *training length from max size of the tweet 132 -> 99.45 percentile length 39 +1*
- *with batch size 512 less sophisticated models overperformed on class 5 and ignored all other classes/ GRU, LSTM and longerRNN have done better, but still worse than with batch size 100. Batch size 100 worked out more stable for all models being able to distinguish classes 5, 4, 1. We'l stop with batch 64*


In [12]:
num_classes = 5
batch_size = 64
epochs=5

In [13]:
x_train, x_test, y_train, y_test = train_test_split(df['processed'], df['Rating'])

In [14]:
text_corpus_train = x_train.values
text_corpus_test = x_test.values

In [15]:
np.percentile([len(i.split()) for i in text_corpus_train], 99.45)

41.0

In [16]:
np.mean([len(i.split()) for i in text_corpus_train])

4.998838259971602

In [17]:
training_length = 40

In [18]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])


In [19]:
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

In [20]:
np.shape(X_train)

(15494, 132)

In [21]:
training_length

132

**Simple RNN**

In [22]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=132,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [23]:
y_train_categ = keras.utils.to_categorical(y_train-1, num_classes)
y_test_categ = keras.utils.to_categorical(y_test-1, num_classes)

In [24]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train_categ,
                    batch_size=batch_size,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
218/218 [==============================] - 15s 62ms/step - loss: 0.8148 - categorical_accuracy: 0.7403 - val_loss: 0.6831 - val_categorical_accuracy: 0.7613
Epoch 2/5
218/218 [==============================] - 13s 62ms/step - loss: 0.6120 - categorical_accuracy: 0.7915 - val_loss: 0.6835 - val_categorical_accuracy: 0.7600


In [25]:
score = model.evaluate(X_test, y_test_categ, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score)

81/81 [==============================] - 1s 16ms/step - loss: 0.6752 - categorical_accuracy: 0.7634


Test score: [0.6752148866653442, 0.7634075284004211]


In [26]:
results_RNN = model.predict(X_test, batch_size=batch_size, verbose=1)

81/81 [==============================] - 1s 15ms/step


In [27]:
def get_classes_df(results, true_classes_values, index=None):
    classes = [np.argmax(x)+1 for x in results]
    if index is not None:
        df_classes = pd.DataFrame({'true': true_classes_values, 'pred': classes},
                                   index=index, columns=['true', 'pred'])
    else:
        df_classes = pd.DataFrame({'true': true_classes_values, 'pred': classes},
                                   columns=['true', 'pred'])
    df_classes['accuracy_tag'] = 0
    df_classes.loc[df_classes['true']==df_classes['pred'], ['accuracy_tag']] = 1
    
    return df_classes

In [28]:
def get_accuracy(df, class_name):
    accuracy_tags = df.loc[(df['true']==class_name), ['accuracy_tag']].value_counts()
    false_tags = df.loc[(df['true']==class_name), ['pred']].value_counts()
    total = 0
    if 1 in accuracy_tags.index:
        if 0 in accuracy_tags.index:
            total = accuracy_tags[1]+accuracy_tags[0]            
            accuracy_percent = accuracy_tags[1] / total
        else:
            total = accuracy_tags[1]            
            accuracy_percent = 100
    else:
        total = accuracy_tags[0]
        accuracy_percent=0
    
    return false_tags, accuracy_percent, total

In [29]:
df_test_preds = get_classes_df(results_RNN, y_test.values, index=y_test.index)

In [30]:
df_test_preds['true'].value_counts()

true
5    3647
1     556
4     534
3     227
2     201
Name: count, dtype: int64

In [31]:
y_train.value_counts()

Rating
5    10939
1     1720
4     1604
3      684
2      547
Name: count, dtype: int64

In [32]:
for class_name in df_test_preds['pred'].value_counts().index:
    false_tags, g_a, total = get_accuracy(df_test_preds, class_name)
    print(f'Accuracy for {class_name} class is {g_a} %')
    if g_a <= 0.4:
        print(f'Number of predicted class tags for {class_name} class in total {total}: \n {false_tags}')

Accuracy for 5 class is 0.9479023855223472 %
Accuracy for 1 class is 0.7482014388489209 %
Accuracy for 4 class is 0.12172284644194757 %
Number of predicted class tags for 4 class in total 534: 
 pred
5       353
1       105
4        65
3         6
2         5
Name: count, dtype: int64
Accuracy for 3 class is 0.022026431718061675 %
Number of predicted class tags for 3 class in total 227: 
 pred
1       124
5        61
4        32
2         5
3         5
Name: count, dtype: int64
Accuracy for 2 class is 0 %
Number of predicted class tags for 2 class in total 201: 
 pred
1       138
5        36
4        23
3         4
Name: count, dtype: int64


*worse than CNN in hw7*

**CNN+RNN**

In [33]:
model_CRNN = Sequential()

model_CRNN.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=64,
              trainable=True,
              mask_zero=True))
model_CRNN.add(Masking(mask_value=0.0))
model_CRNN.add(Conv1D(128, 3))
model_CRNN.add(Activation("relu"))
model_CRNN.add(GlobalMaxPool1D(data_format="channels_first", keepdims=True))
model_CRNN.add(SimpleRNN(64))
model_CRNN.add(Dense(32, activation='relu'))
model_CRNN.add(Dropout(0.5))
model_CRNN.add(Dense(num_classes, activation='softmax'))

model_CRNN.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [34]:
model_CRNN.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 132, 64)           710976    
                                                                 
 masking_1 (Masking)         (None, 132, 64)           0         
                                                                 
 conv1d (Conv1D)             (None, 130, 128)          24704     
                                                                 
 activation (Activation)     (None, 130, 128)          0         
                                                                 
 global_max_pooling1d (Globa  (None, 130, 1)           0         
 lMaxPooling1D)                                                  
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                4224      
                                                      

In [35]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model_CRNN.fit(X_train, y_train_categ,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
218/218 [==============================] - 12s 47ms/step - loss: 0.8985 - categorical_accuracy: 0.7108 - val_loss: 0.7111 - val_categorical_accuracy: 0.7465
Epoch 2/5
218/218 [==============================] - 10s 47ms/step - loss: 0.7054 - categorical_accuracy: 0.7618 - val_loss: 0.6844 - val_categorical_accuracy: 0.7523
Epoch 3/5
218/218 [==============================] - 10s 46ms/step - loss: 0.6236 - categorical_accuracy: 0.7830 - val_loss: 0.6942 - val_categorical_accuracy: 0.7535


In [36]:
score = model_CRNN.evaluate(X_test, y_test_categ, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score)

81/81 [==============================] - 1s 15ms/step - loss: 0.6875 - categorical_accuracy: 0.7636


Test score: [0.687508225440979, 0.7636011838912964]


In [37]:
results_CRNN = model_CRNN.predict(X_test, batch_size=batch_size, verbose=1)

81/81 [==============================] - 1s 15ms/step


In [38]:
df_CRNN_preds = get_classes_df(results_CRNN, y_test.values, index=y_test.index)

In [39]:
df_CRNN_preds.loc[(df_CRNN_preds['true']==5), ['accuracy_tag']].value_counts()

accuracy_tag
1               3479
0                168
Name: count, dtype: int64

In [40]:
for class_name in df_CRNN_preds['pred'].value_counts().index:
    false_tags, g_a, total = get_accuracy(df_CRNN_preds, class_name)
    print(f'Accuracy for {class_name} class is {g_a} %')
    if g_a <= 0.4:
        print(f'Number of predicted class tags for {class_name} class in total {total}: \n {false_tags}')

Accuracy for 5 class is 0.9539347408829175 %
Accuracy for 1 class is 0.8309352517985612 %
Accuracy for 4 class is 0.0056179775280898875 %
Number of predicted class tags for 4 class in total 534: 
 pred
5       381
1       150
4         3
Name: count, dtype: int64


**RNN+CNN**

In [41]:
model_RCNN = Sequential()

model_RCNN.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=64,
              trainable=True,
              mask_zero=True))
#model_RCNN.add(Masking(mask_value=0.0))
model_RCNN.add(SimpleRNN(128))
model_RCNN.add(Reshape((128, 1), input_shape=(128,)))
model_RCNN.add(Conv1D(128, 3))
model_RCNN.add(Activation("relu"))
model_RCNN.add(GlobalMaxPool1D(data_format="channels_first", keepdims=True))
model_RCNN.add(Conv1D(80, 3))
model_RCNN.add(Activation("relu"))
#model_RCNN.add(GlobalMaxPool1D(data_format="channels_first", keepdims=True))
model_RCNN.add(Conv1D(50, 3))
model_RCNN.add(Activation("relu"))
model_RCNN.add(GlobalMaxPool1D())
#model_RCNN.add(GlobalMaxPool1D(data_format="channels_first", keepdims=True))
model_RCNN.add(Dense(32, activation='relu'))
model_RCNN.add(Dropout(0.5))
model_RCNN.add(Dense(num_classes, activation='softmax'))

model_RCNN.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [42]:
model_RCNN.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 132, 64)           710976    
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 128)               24704     
                                                                 
 reshape (Reshape)           (None, 128, 1)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 126, 128)          512       
                                                                 
 activation_1 (Activation)   (None, 126, 128)          0         
                                                                 
 global_max_pooling1d_1 (Glo  (None, 126, 1)           0         
 balMaxPooling1D)                                                
                                                      

In [43]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model_RCNN.fit(X_train, y_train_categ,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
218/218 [==============================] - 22s 92ms/step - loss: 1.0140 - categorical_accuracy: 0.7030 - val_loss: 0.7950 - val_categorical_accuracy: 0.7032
Epoch 2/5
218/218 [==============================] - 20s 90ms/step - loss: 0.7669 - categorical_accuracy: 0.7155 - val_loss: 0.7560 - val_categorical_accuracy: 0.7181
Epoch 3/5
218/218 [==============================] - 20s 90ms/step - loss: 0.6755 - categorical_accuracy: 0.7524 - val_loss: 0.7559 - val_categorical_accuracy: 0.7084
Epoch 4/5
218/218 [==============================] - 19s 89ms/step - loss: 0.6271 - categorical_accuracy: 0.7715 - val_loss: 0.8054 - val_categorical_accuracy: 0.7013


In [44]:
score = model_RCNN.evaluate(X_test, y_test_categ, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score)

81/81 [==============================] - 2s 27ms/step - loss: 0.7858 - categorical_accuracy: 0.7336


Test score: [0.7857568264007568, 0.7335914969444275]


In [45]:
results_RCNN = model_RCNN.predict(X_test, batch_size=batch_size, verbose=1)

81/81 [==============================] - 2s 27ms/step


In [46]:
df_RCNN_preds = get_classes_df(results_RCNN, y_test.values, index=y_test.index)

In [47]:
for class_name in df_RCNN_preds['pred'].value_counts().index:
    false_tags, g_a, total = get_accuracy(df_RCNN_preds, class_name)
    print(f'Accuracy for {class_name} class is {g_a} %')
    if g_a <= 0.4:
        print(f'Number of predicted class tags for {class_name} class in total {total}: \n {false_tags}')

Accuracy for 5 class is 0.8947079791609542 %
Accuracy for 1 class is 0.9226618705035972 %
Accuracy for 4 class is 0.024344569288389514 %
Number of predicted class tags for 4 class in total 534: 
 pred
5       278
1       243
4        13
Name: count, dtype: int64


**Longer RNN**

In [48]:
model_longRNN = Sequential()

model_longRNN.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model_longRNN.add(Masking(mask_value=0.0))
model_longRNN.add(SimpleRNN(128, return_sequences=True))
model_longRNN.add(SimpleRNN(80))
model_longRNN.add(Dense(64, activation='relu'))
model_longRNN.add(Dense(32, activation='relu'))
model_longRNN.add(Dropout(0.5))
model_longRNN.add(Dense(num_classes, activation='softmax'))

model_longRNN.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [49]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model_longRNN.fit(X_train, y_train_categ,
                    batch_size=batch_size,
                    epochs=5,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
218/218 [==============================] - 32s 134ms/step - loss: 0.8559 - categorical_accuracy: 0.7185 - val_loss: 0.6957 - val_categorical_accuracy: 0.7503
Epoch 2/5
218/218 [==============================] - 29s 133ms/step - loss: 0.6739 - categorical_accuracy: 0.7704 - val_loss: 0.6850 - val_categorical_accuracy: 0.7542
Epoch 3/5
218/218 [==============================] - 29s 133ms/step - loss: 0.5560 - categorical_accuracy: 0.8110 - val_loss: 0.7235 - val_categorical_accuracy: 0.7394


In [50]:
score = model_longRNN.evaluate(X_test, y_test_categ, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score)

81/81 [==============================] - 3s 42ms/step - loss: 0.7242 - categorical_accuracy: 0.7518


Test score: [0.7241938710212708, 0.7517908811569214]


In [51]:
results_longRNN = model_longRNN.predict(X_test, batch_size=batch_size, verbose=1)

81/81 [==============================] - 4s 44ms/step


In [52]:
df_test_longRNN = get_classes_df(results_longRNN, y_test.values, index=y_test.index)

In [53]:
for class_name in df_test_longRNN['pred'].value_counts().index:
    false_tags, g_a, total = get_accuracy(df_test_longRNN, class_name)
    print(f'Accuracy for {class_name} class is {g_a} %')
    if g_a <= 0.4:
        print(f'Number of predicted class tags for {class_name} class in total {total}: \n {false_tags} ')

Accuracy for 5 class is 0.945160405812997 %
Accuracy for 1 class is 0.6564748201438849 %
Accuracy for 4 class is 0.10486891385767791 %
Number of predicted class tags for 4 class in total 534: 
 pred
5       356
1       103
4        56
2        11
3         8
Name: count, dtype: int64 
Accuracy for 2 class is 0.03980099502487562 %
Number of predicted class tags for 2 class in total 201: 
 pred
1       130
5        31
4        21
3        11
2         8
Name: count, dtype: int64 
Accuracy for 3 class is 0.030837004405286344 %
Number of predicted class tags for 3 class in total 227: 
 pred
1       103
5        63
4        36
2        18
3         7
Name: count, dtype: int64 


**disbalanced classes is a real problem. I guess if this was a real task to work with this dataset with given instruments I would try to classify each small class (2, 3, 4) separately from all the others together**

**LSTM**

In [54]:
model_LSTM = Sequential()

model_LSTM.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model_LSTM.add(Masking(mask_value=0.0))
model_LSTM.add(LSTM(64, recurrent_dropout=0.2))
model_LSTM.add(Dense(64, activation='relu'))
model_LSTM.add(Dropout(0.5))
model_LSTM.add(Dense(num_classes, activation='softmax'))

model_LSTM.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
early_stopping=EarlyStopping(monitor='val_loss')  


history = model_LSTM.fit(X_train, y_train_categ,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
218/218 [==============================] - 33s 136ms/step - loss: 0.8758 - categorical_accuracy: 0.7238 - val_loss: 0.7025 - val_categorical_accuracy: 0.7503
Epoch 2/5
218/218 [==============================] - 28s 130ms/step - loss: 0.6657 - categorical_accuracy: 0.7726 - val_loss: 0.6667 - val_categorical_accuracy: 0.7594
Epoch 3/5
218/218 [==============================] - 28s 128ms/step - loss: 0.5893 - categorical_accuracy: 0.7915 - val_loss: 0.6820 - val_categorical_accuracy: 0.7613


In [55]:
score = model_LSTM.evaluate(X_test, y_test_categ, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score)

81/81 [==============================] - 2s 22ms/step - loss: 0.6930 - categorical_accuracy: 0.7601


Test score: [0.692980170249939, 0.7601161599159241]


In [56]:
results_LSTM = model_LSTM.predict(X_test, batch_size=batch_size, verbose=1)

81/81 [==============================] - 2s 22ms/step


In [57]:
df_LSTM_preds = get_classes_df(results_LSTM, y_test.values, index=y_test.index)

In [58]:
for class_name in df_LSTM_preds['pred'].value_counts().index:
    false_tags, g_a, total = get_accuracy(df_LSTM_preds, class_name)
    print(f'Accuracy for {class_name} class is {g_a} %')
    if g_a <= 0.4:
        print(f'Number of predicted class tags for {class_name} class: \n {false_tags} from total {total}')

Accuracy for 5 class is 0.9520153550863724 %
Accuracy for 1 class is 0.7392086330935251 %
Accuracy for 4 class is 0.07865168539325842 %
Number of predicted class tags for 4 class: 
 pred
5       390
1       101
4        42
3         1
Name: count, dtype: int64 from total 534
Accuracy for 3 class is 0.004405286343612335 %
Number of predicted class tags for 3 class: 
 pred
1       125
5        75
4        25
2         1
3         1
Name: count, dtype: int64 from total 227
Accuracy for 2 class is 0 %
Number of predicted class tags for 2 class: 
 pred
1       144
5        42
4        15
Name: count, dtype: int64 from total 201


**GRU**

In [59]:
model_GRU = Sequential()

model_GRU.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model_GRU.add(Masking(mask_value=0.0))
model_GRU.add(GRU(64, recurrent_dropout=0.2))
model_GRU.add(Dense(64, activation='relu'))
model_GRU.add(Dropout(0.5))
model_GRU.add(Dense(num_classes, activation='softmax'))

model_GRU.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#early_stopping=EarlyStopping(monitor='val_loss')  

history = model_GRU.fit(X_train, y_train_categ,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/5
218/218 [==============================] - 27s 109ms/step - loss: 0.8302 - categorical_accuracy: 0.7303 - val_loss: 0.6969 - val_categorical_accuracy: 0.7548
Epoch 2/5
218/218 [==============================] - 23s 107ms/step - loss: 0.6382 - categorical_accuracy: 0.7795 - val_loss: 0.6764 - val_categorical_accuracy: 0.7600
Epoch 3/5
218/218 [==============================] - 23s 107ms/step - loss: 0.5547 - categorical_accuracy: 0.8008 - val_loss: 0.6988 - val_categorical_accuracy: 0.7490


In [60]:
score = model_GRU.evaluate(X_test, y_test_categ, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score)

81/81 [==============================] - 2s 19ms/step - loss: 0.6978 - categorical_accuracy: 0.7576


Test score: [0.6978428363800049, 0.7575992345809937]


*Test score after 3 epochs*   
Test score: [0.7368313670158386, 0.7589545249938965]

In [61]:
results_GRU = model_GRU.predict(X_test, batch_size=batch_size, verbose=1)

81/81 [==============================] - 2s 19ms/step


In [62]:
df_GRU_preds = get_classes_df(results_GRU, y_test.values, index=y_test.index)

In [63]:
for class_name in df_GRU_preds['pred'].value_counts().index:
    false_tags, g_a, total = get_accuracy(df_GRU_preds, class_name)
    print(f'Accuracy for {class_name} class is {g_a} %')
    if g_a <= 0.4:
        print(f'Number of predicted class tags for {class_name} class: \n {false_tags} from total {total}')

Accuracy for 5 class is 0.9366602687140115 %
Accuracy for 1 class is 0.6888489208633094 %
Accuracy for 4 class is 0.12734082397003746 %
Number of predicted class tags for 4 class: 
 pred
5       340
1        87
4        68
3        39
Name: count, dtype: int64 from total 534
Accuracy for 3 class is 0.1894273127753304 %
Number of predicted class tags for 3 class: 
 pred
1       85
5       54
3       43
4       41
2        4
Name: count, dtype: int64 from total 227
Accuracy for 2 class is 0.014925373134328358 %
Number of predicted class tags for 2 class: 
 pred
1       120
5        31
4        28
3        19
2         3
Name: count, dtype: int64 from total 201


**After 3 epochs (early_stop = 'validation loss') with batch size 100**   
Accuracy for 5 class is 0.9540704070407041 %   
Accuracy for 1 class is 0.7403314917127072 %   
Accuracy for 4 class is 0.09404990403071017 %   
Number of predicted class tags for 4 class:    
 pred   
5       357   
1       115   
4        49   
Name: count, dtype: int64   
**Classes 2 and 3 have 0% accuracy**